In [ ]:
# ============================================================
#  HXAI EXPERIMENT REPRODUCTION SCRIPT (FULL SINGLE SCRIPT)
#  Generates: SHAP, DP Aggregation, Utility, Ranking Stability,
#  Leakage Curves, RMSE, Baselines, Plots, Metrics CSV
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from numpy.linalg import norm
from scipy.stats import spearmanr
import shap
import requests
import seaborn as sns
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

sns.set(style="whitegrid")

# ------------------------------------------
# GLOBAL PARAMETERS
# ------------------------------------------
k = 50       # households
d = 7        # appliances (Features)
T = 168      # hours (1 week of time series data)
epsilons = [0.25, 0.5, 1, 2, 4, 8]
np.random.seed(42)

print("Starting HXAI Experiment Pipeline...")

# ------------------------------------------
# SECTION 1 — Simulated Household Data
# ------------------------------------------

print("Generating appliance data...")

# Average power consumption (kW) for 7 simulated appliances
P = np.array([3.0, 2.0, 0.8, 1.2, 1.0, 0.15, 0.2])

def activation_prob(hour):
    """
    Defines the probability of each appliance being active based on the hour of the day.
    Appliance index: 0-Washer, 1-Dryer, 2-Dishwasher, 3-Oven, 4-AC, 5-Lights, 6-Electronics
    """
    # Normalize hour to 0-23 for daily pattern
    h = hour % 24

    if 6 <= h <= 9:  # Morning peak
        # High probability for Washer, Dryer, Lights, Electronics
        return np.array([0.9, 0.7, 0.1, 0.05, 0.05, 1.0, 0.8])
    elif 17 <= h <= 21:  # Evening peak
        # High probability for Washer, Dishwasher, Oven, AC, Lights, Electronics
        return np.array([0.8, 0.4, 0.4, 0.3, 0.6, 1.0, 0.9])
    else:  # Off-peak
        return np.array([0.5, 0.3, 0.05, 0.1, 0.1, 1.0, 0.5])

households = []

for i in range(k):
    # X: (T, d) matrix of hourly appliance consumption (Features)
    # y: (T,) vector of total household consumption (Target)
    X = np.zeros((T, d))
    for t in range(T):
        probs = activation_prob(t)
        # Randomly decide if appliance is active (1) or inactive (0)
        active = np.random.binomial(1, probs)
        # Add small normally distributed noise to consumption
        noise = np.random.normal(0, 0.05, d)
        X[t] = P * active + noise

    y = X.sum(axis=1) # Total consumption is the sum of appliance consumption
    households.append((X, y))


# ------------------------------------------
# SECTION 2 — Norwegian Price Pattern (NO1)
# ------------------------------------------

print("Fetching Norwegian price pattern (NO1)...")

def fetch_no1_prices():
    """
    Fetches real-time hourly NordPool energy prices for the NO1 (Oslo) zone
    via the Open-Meteo API.
    """
    url = "https://api.open-meteo.com/v1/nordpool?zone=NO1&hourly=price"

    # Define a synthetic fallback in case the API call fails
    synthetic_price_pattern = pd.DataFrame({
        "price": np.sin(np.linspace(0, 10, T))*10 + 50
    })

    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        data = r.json()

        # Ensure 'hourly' and 'price' data exists
        if "hourly" not in data or "price" not in data["hourly"]:
            print("⚠️ NordPool API response missing 'price' data. Using synthetic pattern.")
            return synthetic_price_pattern

        df = pd.DataFrame({
            "time": pd.to_datetime(data["hourly"]["time"]),
            "price": data["hourly"]["price"]
        })

        # Take the last T hours of available data
        df = df.tail(T).reset_index(drop=True)

        # Fill any potential missing values (NaNs) by carrying the backward fill
        # (using the next available price for the missing hour)
        df["price"] = df["price"].fillna(method="bfill")

        if len(df) < T:
             print(f"⚠️ NordPool API returned only {len(df)} data points. Using synthetic pattern.")
             return synthetic_price_pattern

        return df

    except requests.exceptions.RequestException as e:
        print(f"⚠️ Could not fetch NordPool API: {e}. Using synthetic price pattern.")
        return synthetic_price_pattern

price_df = fetch_no1_prices()
price_pattern = price_df["price"].values


# ------------------------------------------
# SECTION 3 — Local Model + SHAP
# ------------------------------------------

print("Training local models and computing SHAP...")

local_models = []
local_shap_summaries = []

# Appliance names for clear visualization and output
feature_names = [f'Appliance_{i+1}' for i in range(d)]

for i in range(k):
    X, y = households[i]

    # Gradient Boosting Regressor is the local energy forecasting model
    model = GradientBoostingRegressor(random_state=42)
    model.fit(X, y)

    # SHAP Explainer for tree-based models
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)

    # S_i: The sum of SHAP values over all T time steps for household i.
    # This represents the total feature importance (summary explanation).
    S_i = shap_values.sum(axis=0)
    local_models.append(model)
    local_shap_summaries.append(S_i)

# The ground truth for the zonal explanation is the aggregate of all local SHAP summaries
fed_shap = np.sum(local_shap_summaries, axis=0)


# ------------------------------------------
# SECTION 4 — DP Aggregation (HXAI)
# ------------------------------------------

print("Applying DP aggregation...")

def laplace_noise(scale, size):
    """Generates random noise from a Laplace distribution."""
    return np.random.laplace(0, scale, size)

dp_results = {}

for eps in epsilons:
    dp_summaries = []

    for S in local_shap_summaries:
        # L1-sensitivity (local clipping) is set to the L-infinity norm (max absolute value)
        # of the feature importance vector, S.
        sensitivity = norm(S, ord=1) # Using L1 norm as a common choice for L1 sensitivity
        # For a Laplace mechanism: scale = sensitivity / epsilon
        scale = sensitivity / eps
        # Add Laplace noise to the local explanation S
        noisy = S + laplace_noise(scale, d)
        dp_summaries.append(noisy)

    # S_zonal: The differentially private aggregate explanation
    S_zonal = np.sum(dp_summaries, axis=0)
    dp_results[eps] = S_zonal


# ------------------------------------------
# SECTION 5 — Baselines (Non-DP Comparisons)
# ------------------------------------------

print("Computing baselines...")

# Global SHAP (Centralized Non-Private)
X_global = np.vstack([h[0] for h in households])
y_global = np.hstack([h[1] for h in households])

global_model = GradientBoostingRegressor(random_state=42)
global_model.fit(X_global, y_global)

global_exp = shap.TreeExplainer(global_model)
shap_global = global_exp.shap_values(X_global)
baseline_global = shap_global.sum(axis=0) # Aggregate SHAP from a single global model

# Local DP (Data-level perturbation, less effective for aggregation)
ldp_summaries = []
for X, y in households:
    # Perturb the *data* before training (DP-level on input data X)
    X_ldp = X + np.random.laplace(0, 0.5, X.shape)
    model_ldp = GradientBoostingRegressor(random_state=42)
    model_ldp.fit(X_ldp, y)
    exp_ldp = shap.TreeExplainer(model_ldp)
    vals_ldp = exp_ldp.shap_values(X_ldp)
    ldp_summaries.append(vals_ldp.sum(axis=0))
baseline_ldp = np.sum(ldp_summaries, axis=0) # Aggregate SHAP from perturbed-data models

# DP-SGD (Simple approximation: add noise to the non-private global model SHAP)
# This simulates the final output of a complex DP-SGD training process.
dp_global = baseline_global + laplace_noise(1.0, d)


# ------------------------------------------
# SECTION 6 — Metrics
# ------------------------------------------

print("Computing metrics...")

def utility(true, noisy):
    """
    Computes Cosine Similarity between the true (non-private) and noisy
    (DP-aggregated) explanation vectors. A value closer to 1 is better.
    """
    # Protect against division by zero if either vector has zero norm
    true_norm = norm(true)
    noisy_norm = norm(noisy)
    if true_norm == 0 or noisy_norm == 0:
        return 0.0

    return np.dot(true, noisy) / (true_norm * noisy_norm)

U = {eps: utility(fed_shap, dp_results[eps]) for eps in epsilons}

# Ranking Stability (Spearman's rank correlation coefficient)
# Measures how well the noisy feature ranking matches the true ranking.
rank_stab = {}
for eps in epsilons:
    # spearmanr returns (correlation, p-value)
    rho, _ = spearmanr(fed_shap, dp_results[eps])
    rank_stab[eps] = rho if not np.isnan(rho) else 0.0 # Handle case of constant vectors

def leakage_curve(eps, max_queries=20):
    """
    Simulates the growth of privacy leakage (epsilon budget exhaustion)
    under sequential queries in a DP-compliant system.
    """
    leakage = []
    total = 0
    # Simulate an accountant capping the total accumulated epsilon
    max_cap = 10.0
    for _ in range(max_queries):
        total += eps
        leakage.append(min(total, max_cap))
    return leakage

leakages = {eps: leakage_curve(eps) for eps in epsilons}

# Root Mean Squared Error (RMSE)
# Measures the raw error magnitude between true and noisy explanations.
rmse = {}
for eps in epsilons:
    rmse[eps] = np.sqrt(mean_squared_error(fed_shap, dp_results[eps]))


# ------------------------------------------
# SECTION 7 — Generate Plots
# ------------------------------------------

print("Generating plots...")

# Utility Plot
plt.figure(figsize=(7,5))
plt.plot(epsilons, [U[e] for e in epsilons], marker='o', color='tab:blue')
plt.axhline(utility(fed_shap, baseline_global), color='tab:red', linestyle='--', label='Global SHAP (Non-Private)')
plt.xscale('log', base=2)
plt.xlabel("Privacy Budget ($\epsilon$)", fontsize=12)
plt.ylabel("Utility (Cosine Similarity)", fontsize=12)
plt.title("Explanation Utility vs Privacy Budget", fontsize=14)
plt.legend()
plt.tight_layout()
plt.savefig("utility_curve.png")
plt.close()

# Ranking Stability Plot
plt.figure(figsize=(7,5))
plt.plot(epsilons, [rank_stab[e] for e in epsilons], marker='s', color='tab:green')
plt.axhline(spearmanr(fed_shap, baseline_global)[0], color='tab:red', linestyle='--', label='Global SHAP (Non-Private)')
plt.xscale('log', base=2)
plt.xlabel("Privacy Budget ($\epsilon$)", fontsize=12)
plt.ylabel("Spearman Correlation ($\u03C1$)", fontsize=12)
plt.title("SHAP Ranking Stability vs Privacy Budget", fontsize=14)
plt.legend()
plt.tight_layout()
plt.savefig("rank_stability.png")
plt.close()

# Leakage Plot
plt.figure(figsize=(7,5))
for eps in epsilons:
    plt.plot(leakages[eps], label=f"$\epsilon$={eps}")
plt.xlabel("Query Number", fontsize=12)
plt.ylabel("Cumulative Leakage ($\sum \epsilon$)", fontsize=12)
plt.title("Leakage Growth Under Sequential Queries", fontsize=14)
plt.legend(title='Epsilon per Query')
plt.tight_layout()
plt.savefig("leakage_curve.png")
plt.close()

# RMSE Plot
plt.figure(figsize=(7,5))
plt.plot(epsilons, [rmse[e] for e in epsilons], marker='d', color='tab:orange')
plt.axhline(np.sqrt(mean_squared_error(fed_shap, baseline_global)), color='tab:red', linestyle='--', label='Global SHAP (Non-Private)')
plt.xscale('log', base=2)
plt.xlabel("Privacy Budget ($\epsilon$)", fontsize=12)
plt.ylabel("RMSE", fontsize=12)
plt.title("Forecasting Error (RMSE) vs Privacy Budget", fontsize=14)
plt.legend()
plt.tight_layout()
plt.savefig("rmse_curve.png")
plt.close()

# Display Feature Importance (for one epsilon)
plt.figure(figsize=(8, 6))
eps_to_plot = 1.0 # Choose a representative epsilon
results = pd.DataFrame({
    'Appliance': feature_names,
    'True SHAP ($\sum S_i$)': fed_shap,
    f'DP-SHAP ($\epsilon$={eps_to_plot})': dp_results[eps_to_plot],
    'Global SHAP': baseline_global
}).set_index('Appliance')

results[['True SHAP ($\sum S_i$)', f'DP-SHAP ($\epsilon$={eps_to_plot})', 'Global SHAP']].plot(
    kind='bar', figsize=(10, 6), width=0.8, color=['tab:red', 'tab:green', 'tab:blue']
)
plt.title(f'Comparison of True vs. DP-Aggregated Feature Importance ($\epsilon$={eps_to_plot})')
plt.ylabel('Aggregate SHAP Value')
plt.xlabel('Appliance')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Explanation Type')
plt.tight_layout()
plt.savefig("feature_importance_comparison.png")
plt.close()


# ------------------------------------------
# SECTION 8 — Save CSV
# ------------------------------------------

print("Saving metrics to CSV...")

metrics_df = pd.DataFrame({
    "epsilon": epsilons,
    "utility": [U[e] for e in epsilons],
    "rank_stab": [rank_stab[e] for e in epsilons],
    "rmse": [rmse[e] for e in epsilons]
})
metrics_df.to_csv("hxai_metrics.csv", index=False)

# Add Baselines to a separate CSV
baselines_df = pd.DataFrame({
    "Appliance": feature_names,
    "Fed SHAP (Ground Truth)": fed_shap,
    "Global SHAP (Centralized)": baseline_global,
    "LDP (Data Perturbation)": baseline_ldp,
    "DP-SGD Approx": dp_global
})
baselines_df.to_csv("hxai_baselines.csv", index=False)


print("\nAll experiments completed successfully!")
print("Generated files:")
print("- utility_curve.png")
print("- rank_stability.png")
print("- leakage_curve.png")
print("- rmse_curve.png")
print("- feature_importance_comparison.png")
print("- hxai_metrics.csv")
print("- hxai_baselines.csv")

<>:294: SyntaxWarning: invalid escape sequence '\e'
<>:307: SyntaxWarning: invalid escape sequence '\e'
<>:318: SyntaxWarning: invalid escape sequence '\e'
<>:320: SyntaxWarning: invalid escape sequence '\s'
<>:332: SyntaxWarning: invalid escape sequence '\e'
<>:345: SyntaxWarning: invalid escape sequence '\s'
<>:346: SyntaxWarning: invalid escape sequence '\e'
<>:350: SyntaxWarning: invalid escape sequence '\s'
<>:350: SyntaxWarning: invalid escape sequence '\e'
<>:353: SyntaxWarning: invalid escape sequence '\e'
<>:294: SyntaxWarning: invalid escape sequence '\e'
<>:307: SyntaxWarning: invalid escape sequence '\e'
<>:318: SyntaxWarning: invalid escape sequence '\e'
<>:320: SyntaxWarning: invalid escape sequence '\s'
<>:332: SyntaxWarning: invalid escape sequence '\e'
<>:345: SyntaxWarning: invalid escape sequence '\s'
<>:346: SyntaxWarning: invalid escape sequence '\e'
<>:350: SyntaxWarning: invalid escape sequence '\s'
<>:350: SyntaxWarning: invalid escape sequence '\e'
<>:353: Synt

Starting HXAI Experiment Pipeline...
Generating appliance data...
Fetching Norwegian price pattern (NO1)...
⚠️ Could not fetch NordPool API: 404 Client Error: Not Found for url: https://api.open-meteo.com/v1/nordpool?zone=NO1&hourly=price. Using synthetic price pattern.
Training local models and computing SHAP...
Applying DP aggregation...
Computing baselines...
Computing metrics...
Generating plots...
Saving metrics to CSV...

All experiments completed successfully!
Generated files:
- utility_curve.png
- rank_stability.png
- leakage_curve.png
- rmse_curve.png
- feature_importance_comparison.png
- hxai_metrics.csv
- hxai_baselines.csv


<Figure size 800x600 with 0 Axes>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import math
import zipfile
import argparse
import warnings
from dataclasses import dataclass
from typing import Tuple, List

import numpy as np
import pandas as pd

from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

import shap
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


# -----------------------------
# Core helpers
# -----------------------------
def set_seed(seed: int = 42):
    np.random.seed(seed)

def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def cosine_similarity(a: np.ndarray, b: np.ndarray, eps: float = 1e-12) -> float:
    a = np.asarray(a).reshape(-1)
    b = np.asarray(b).reshape(-1)
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) + eps
    return float(np.dot(a, b) / denom)

def laplace_noise(scale: float, size: int) -> np.ndarray:
    return np.random.laplace(loc=0.0, scale=scale, size=size)

def rankdata_desc(v: np.ndarray) -> np.ndarray:
    order = np.argsort(-np.abs(v))
    ranks = np.empty_like(order)
    ranks[order] = np.arange(1, len(v) + 1)
    return ranks

def spearman_rank_corr(a: np.ndarray, b: np.ndarray) -> float:
    ra = rankdata_desc(a)
    rb = rankdata_desc(b)
    rho, _ = spearmanr(ra, rb)
    return float(rho)

def download_url(url: str, out_path: str, timeout: int = 90):
    """
    Minimal downloader using stdlib only (works in Colab too).
    """
    import urllib.request
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req, timeout=timeout) as r:
        data = r.read()
    with open(out_path, "wb") as f:
        f.write(data)

def safe_read_csv(path: str, **kwargs) -> pd.DataFrame:
    return pd.read_csv(path, **kwargs)


@dataclass
class DatasetSpec:
    name: str
    target_col: str
    feature_cols: List[str]
    time_col: str = None


# -----------------------------
# Fetch datasets (auto online)
# -----------------------------
def fetch_uci_appliances(data_dir: str) -> Tuple[pd.DataFrame, DatasetSpec]:
    """
    UCI Appliances energy prediction:
    https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv
    """
    ensure_dir(data_dir)
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv"
    csv_path = os.path.join(data_dir, "energydata_complete.csv")
    if not os.path.exists(csv_path):
        print(f"[download] {url}")
        download_url(url, csv_path)

    df = safe_read_csv(csv_path)
    time_col = "date"
    target_col = "Appliances"
    df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
    df = df.dropna(subset=[time_col]).sort_values(time_col).reset_index(drop=True)

    feature_cols = [c for c in df.columns if c not in [time_col, target_col]]
    spec = DatasetSpec(name="uci_appliances", target_col=target_col, feature_cols=feature_cols, time_col=time_col)
    return df, spec


def fetch_uci_household_power(data_dir: str) -> Tuple[pd.DataFrame, DatasetSpec]:
    """
    UCI Individual household electric power consumption:
    https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip
    """
    ensure_dir(data_dir)
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip"
    zip_path = os.path.join(data_dir, "household_power_consumption.zip")
    if not os.path.exists(zip_path):
        print(f"[download] {url}")
        download_url(url, zip_path)

    txt_path = os.path.join(data_dir, "household_power_consumption.txt")
    if not os.path.exists(txt_path):
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(data_dir)

    df = safe_read_csv(
        txt_path,
        sep=";",
        na_values=["?"],
        low_memory=False
    )

    df["DateTime"] = pd.to_datetime(df["Date"] + " " + df["Time"], errors="coerce", dayfirst=True)
    df = df.dropna(subset=["DateTime"]).sort_values("DateTime").reset_index(drop=True)

    # numeric conversion
    for c in df.columns:
        if c not in ["Date", "Time", "DateTime"]:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    df = df.dropna().reset_index(drop=True)

    target_col = "Global_active_power"
    time_col = "DateTime"
    feature_cols = [c for c in df.columns if c not in [target_col, time_col, "Date", "Time"]]
    spec = DatasetSpec(name="uci_household_power", target_col=target_col, feature_cols=feature_cols, time_col=time_col)
    return df, spec


# -----------------------------
# Supervised lag task
# -----------------------------
def make_supervised_lagged(
    df: pd.DataFrame,
    spec: DatasetSpec,
    lags: int = 6,
    horizon: int = 1
) -> Tuple[pd.DataFrame, pd.Series, List[str]]:
    df = df.copy()
    if spec.time_col is not None:
        df = df.sort_values(spec.time_col).reset_index(drop=True)

    if spec.time_col is not None:
        t = pd.to_datetime(df[spec.time_col])
        df["hour"] = t.dt.hour
        df["dayofweek"] = t.dt.dayofweek
        df["month"] = t.dt.month
        base_features = spec.feature_cols + ["hour", "dayofweek", "month"]
    else:
        base_features = spec.feature_cols

    X = pd.DataFrame(index=df.index)
    for col in base_features:
        for L in range(1, lags + 1):
            X[f"{col}_lag{L}"] = df[col].shift(L)

    y = df[spec.target_col].shift(-horizon)

    valid = X.notnull().all(axis=1) & y.notnull()
    X = X.loc[valid].reset_index(drop=True)
    y = y.loc[valid].reset_index(drop=True)
    return X, y, list(X.columns)


# -----------------------------
# HXAI pipeline
# -----------------------------
def split_into_households_contiguous(X: pd.DataFrame, y: pd.Series, k: int):
    n = len(X)
    block = n // k
    chunks = []
    for i in range(k):
        a = i * block
        b = (i + 1) * block if i < k - 1 else n
        chunks.append((X.iloc[a:b].reset_index(drop=True), y.iloc[a:b].reset_index(drop=True)))
    return chunks

def train_tree_model(X_train: np.ndarray, y_train: np.ndarray, seed: int = 42):
    model = HistGradientBoostingRegressor(
        max_depth=6,
        learning_rate=0.08,
        max_iter=300,
        random_state=seed
    )
    model.fit(X_train, y_train)
    return model

def shap_summary_vector(model, X_bg: np.ndarray, X_eval: np.ndarray) -> np.ndarray:
    """
    Robust SHAP summary computation for tree models.

    - Uses interventional perturbation (safe for partial background coverage)
    - Disables additivity check at shap_values() call
      (required for HistGradientBoosting + interventional SHAP)
    """
    explainer = shap.TreeExplainer(
        model,
        data=X_bg,
        feature_perturbation="interventional"
    )

    shap_vals = explainer.shap_values(
        X_eval,
        check_additivity=False
    )

    shap_vals = np.asarray(shap_vals)
    return np.mean(np.abs(shap_vals), axis=0)



def hxai_zonal_aggregate(local_summaries: List[np.ndarray], epsilons: List[float], sensitivity_l1: float) -> np.ndarray:
    d = local_summaries[0].shape[0]
    zonal = np.zeros(d, dtype=float)
    for Si, eps in zip(local_summaries, epsilons):
        scale = sensitivity_l1 / max(eps, 1e-12)
        zonal += Si + laplace_noise(scale=scale, size=d)
    return zonal

def central_dp_baseline(X: np.ndarray, y: np.ndarray, sensitivity_l1: float, epsilon: float, seed: int = 42):
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, shuffle=False)
    model = train_tree_model(Xtr, ytr, seed=seed)
    preds = model.predict(Xte)
    metrics = {"mse": float(mean_squared_error(yte, preds)), "r2": float(r2_score(yte, preds))}

    bg = Xtr[np.linspace(0, len(Xtr) - 1, num=min(512, len(Xtr))).astype(int)]
    evalX = Xte[np.linspace(0, len(Xte) - 1, num=min(1024, len(Xte))).astype(int)]

    S = shap_summary_vector(model, bg, evalX)
    scale = sensitivity_l1 / max(epsilon, 1e-12)
    return S + laplace_noise(scale=scale, size=S.shape[0]), metrics

def negotiate_epsilons(local_summaries: List[np.ndarray], epsilon_total: float, eps_min: float, eps_max: float, temperature: float = 0.6):
    norms = np.array([np.linalg.norm(Si) for Si in local_summaries], dtype=float)
    norms = np.maximum(norms, 1e-12)
    w = norms ** temperature
    w = w / np.sum(w)

    eps = epsilon_total * w
    eps = np.clip(eps, eps_min, eps_max)

    # renormalize w/ clamps
    for _ in range(10):
        s = float(np.sum(eps))
        if s <= 1e-12:
            eps[:] = epsilon_total / len(eps)
            break
        eps *= (epsilon_total / s)
        eps = np.clip(eps, eps_min, eps_max)
    return eps.tolist()

def run_dataset_experiment(df: pd.DataFrame, spec: DatasetSpec, lags: int, horizon: int,
                           k_households: int, eps_grid: List[float], sensitivity_l1: float, seed: int = 42):

    set_seed(seed)
    Xdf, y, feat_cols = make_supervised_lagged(df, spec, lags=lags, horizon=horizon)

    X = Xdf.values.astype(float)
    yv = y.values.astype(float)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    chunks = split_into_households_contiguous(pd.DataFrame(X), pd.Series(yv), k=k_households)

    local_summaries = []
    local_pred_metrics = []
    d = X.shape[1]

    for i, (Xi, yi) in enumerate(chunks):
        Xi = Xi.values.astype(float)
        yi = yi.values.astype(float)
        Xtr, Xte, ytr, yte = train_test_split(Xi, yi, test_size=0.2, shuffle=False)
        model = train_tree_model(Xtr, ytr, seed=seed + i)

        preds = model.predict(Xte)
        local_pred_metrics.append((float(mean_squared_error(yte, preds)), float(r2_score(yte, preds))))

        bg = Xtr[np.linspace(0, len(Xtr) - 1, num=min(256, len(Xtr))).astype(int)]
        evalX = Xte[np.linspace(0, len(Xte) - 1, num=min(512, len(Xte))).astype(int)]
        local_summaries.append(shap_summary_vector(model, bg, evalX))

    S_true = np.sum(np.stack(local_summaries, axis=0), axis=0)

    rows = []
    for eps in eps_grid:
        # HXAI naive
        S_same = hxai_zonal_aggregate(local_summaries, [eps] * k_households, sensitivity_l1)

        # Special scheme (negotiated eps) with same total budget
        eps_total = k_households * eps
        eps_list = negotiate_epsilons(
            local_summaries=local_summaries,
            epsilon_total=eps_total,
            eps_min=max(0.05, 0.10 * eps),
            eps_max=10.0 * eps,
            temperature=0.6
        )
        S_neg = hxai_zonal_aggregate(local_summaries, eps_list, sensitivity_l1)

        # Central DP baseline
        S_central, central_metrics = central_dp_baseline(X, yv, sensitivity_l1, eps, seed=seed)

        for method, S_hat in [
            ("HXAI_same_eps", S_same),
            ("HXAI_negotiated_eps", S_neg),
            ("Central_DP_SHAP", S_central),
        ]:
            rows.append({
                "dataset": spec.name,
                "method": method,
                "epsilon": eps,
                "utility_cosine": cosine_similarity(S_true, S_hat),
                "rank_spearman": spearman_rank_corr(S_true, S_hat),
                "mean_abs_error": float(np.mean(np.abs(S_true - S_hat))),
                "d_features": d,
                "k_households": k_households,
                "sensitivity_l1": sensitivity_l1,
                "local_avg_mse": float(np.mean([m for m, _ in local_pred_metrics])),
                "local_avg_r2": float(np.mean([r for _, r in local_pred_metrics])),
                "central_mse": central_metrics.get("mse", np.nan),
                "central_r2": central_metrics.get("r2", np.nan),
            })

    return pd.DataFrame(rows)


def plot_curves(res: pd.DataFrame, out_dir: str, dataset_name: str):
    ensure_dir(out_dir)

    def _plot(metric: str, ylabel: str, fname: str):
        plt.figure()
        for method in sorted(res["method"].unique()):
            rr = res[res["method"] == method].sort_values("epsilon")
            plt.plot(rr["epsilon"], rr[metric], marker="o", label=method)
        plt.xscale("log")
        plt.xlabel("epsilon (log scale)")
        plt.ylabel(ylabel)
        plt.title(f"{dataset_name}: {ylabel} vs epsilon")
        plt.grid(True, which="both", alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, fname), dpi=220)
        plt.close()

    _plot("utility_cosine", "Utility (cosine similarity)", f"{dataset_name}_utility_vs_epsilon.png")
    _plot("rank_spearman", "Ranking stability (Spearman)", f"{dataset_name}_rank_vs_epsilon.png")
    _plot("mean_abs_error", "Mean abs error", f"{dataset_name}_mae_vs_epsilon.png")


def main(args):
    set_seed(args.seed)
    ensure_dir(args.out_dir)
    ensure_dir(args.data_dir)

    eps_grid = np.logspace(math.log10(args.eps_min), math.log10(args.eps_max), args.eps_points).tolist()

    all_results = []

    df1, spec1 = fetch_uci_appliances(args.data_dir)
    res1 = run_dataset_experiment(df1, spec1, args.lags, args.horizon, args.k_households, eps_grid, args.sensitivity_l1, seed=args.seed)
    plot_curves(res1, args.out_dir, spec1.name)
    all_results.append(res1)

    df2, spec2 = fetch_uci_household_power(args.data_dir)
    res2 = run_dataset_experiment(df2, spec2, args.lags, args.horizon, args.k_households, eps_grid, args.sensitivity_l1, seed=args.seed)
    plot_curves(res2, args.out_dir, spec2.name)
    all_results.append(res2)

    res_all = pd.concat(all_results, axis=0).reset_index(drop=True)
    csv_path = os.path.join(args.out_dir, "results_realdata_hxai.csv")
    res_all.to_csv(csv_path, index=False)
    print(f"[saved] {csv_path}")

    print("\nQuick check (best utility per dataset/method):")
    print(
        res_all.sort_values("utility_cosine", ascending=False)
              .groupby(["dataset", "method"], as_index=False)
              .head(1)[["dataset","method","epsilon","utility_cosine","rank_spearman","mean_abs_error"]]
              .to_string(index=False)
    )

    print("\nDone. Check output PNGs + CSV in:", args.out_dir)


def build_argparser():
    ap = argparse.ArgumentParser(add_help=True)
    ap.add_argument("--out_dir", type=str, default="hxai_realdata_out")
    ap.add_argument("--data_dir", type=str, default="hxai_realdata_data")
    ap.add_argument("--k_households", type=int, default=20)
    ap.add_argument("--lags", type=int, default=6)
    ap.add_argument("--horizon", type=int, default=1)
    ap.add_argument("--seed", type=int, default=42)
    ap.add_argument("--eps_min", type=float, default=0.1)
    ap.add_argument("--eps_max", type=float, default=20.0)
    ap.add_argument("--eps_points", type=int, default=10)
    ap.add_argument("--sensitivity_l1", type=float, default=1.0)
    return ap
def make_supervised_lagged(df, spec, lags=6, horizon=1):
    df = df.copy()
    if spec.time_col is not None:
        df = df.sort_values(spec.time_col).reset_index(drop=True)

    if spec.time_col is not None:
        t = pd.to_datetime(df[spec.time_col])
        base = df[spec.feature_cols].copy()
        base["hour"] = t.dt.hour
        base["dayofweek"] = t.dt.dayofweek
        base["month"] = t.dt.month
    else:
        base = df[spec.feature_cols].copy()

    lagged_blocks = []
    colnames = []

    for L in range(1, lags + 1):
        shifted = base.shift(L)
        shifted.columns = [f"{c}_lag{L}" for c in shifted.columns]
        lagged_blocks.append(shifted)
        colnames.extend(shifted.columns)

    X = pd.concat(lagged_blocks, axis=1)
    y = df[spec.target_col].shift(-horizon)

    valid = X.notnull().all(axis=1) & y.notnull()
    X = X.loc[valid].reset_index(drop=True)
    y = y.loc[valid].reset_index(drop=True)

    return X, y, colnames


if __name__ == "__main__":
    parser = build_argparser()

    # ✅ KEY FIX: ignore unknown args injected by Jupyter/Colab (e.g., "-f kernel.json")
    args, _unknown = parser.parse_known_args()

    main(args)


 95%|=================== | 975/1024 [00:16<00:00]       

[saved] hxai_realdata_out/results_realdata_hxai.csv

Quick check (best utility per dataset/method):
            dataset              method   epsilon  utility_cosine  rank_spearman  mean_abs_error
     uci_appliances       HXAI_same_eps 20.000000        0.999969       0.999296        0.234703
     uci_appliances HXAI_negotiated_eps 20.000000        0.999957       0.999140        0.286703
uci_household_power HXAI_negotiated_eps 20.000000        0.992340       0.503640        0.206520
uci_household_power       HXAI_same_eps 20.000000        0.991642       0.332342        0.243742
uci_household_power     Central_DP_SHAP 20.000000        0.776633       0.029312        0.440966
     uci_appliances     Central_DP_SHAP  1.898235        0.652329       0.437555       29.378640

Done. Check output PNGs + CSV in: hxai_realdata_out


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
HXAI real-data experiments + qualitative visualization + seed-averaged confidence bands.

Datasets fetched online (no keys):
  1) UCI Appliances energy (energydata_complete.csv)
  2) UCI Household Power consumption (household_power_consumption.zip)
  3) UCI Bike Sharing Hourly (Bike-Sharing-Dataset.zip)

Outputs:
  out_dir/
    results_per_seed.csv
    results_agg_meanstd.csv
    curves_<dataset>_{utility,rank,mae}.(png|pdf)
    qual_<dataset>_bars_eps{...}.(png|pdf)
    qual_<dataset>_ranktraj_<method>.(png|pdf)
"""

import os
import math
import zipfile
import argparse
import warnings
from dataclasses import dataclass
from typing import Tuple, List, Dict, Optional

import numpy as np
import pandas as pd

from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

import shap
import matplotlib.pyplot as plt


# -----------------------------
# Warnings
# -----------------------------
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


# -----------------------------
# Utilities
# -----------------------------
def set_seed(seed: int):
    np.random.seed(seed)

def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def download_url(url: str, out_path: str, timeout: int = 120):
    import urllib.request
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req, timeout=timeout) as r:
        data = r.read()
    with open(out_path, "wb") as f:
        f.write(data)

def cosine_similarity(a: np.ndarray, b: np.ndarray, eps: float = 1e-12) -> float:
    a = np.asarray(a).reshape(-1)
    b = np.asarray(b).reshape(-1)
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) + eps
    return float(np.dot(a, b) / denom)

def laplace_noise(scale: float, size: int) -> np.ndarray:
    return np.random.laplace(loc=0.0, scale=scale, size=size)

def rankdata_desc(v: np.ndarray) -> np.ndarray:
    order = np.argsort(-np.abs(v))
    ranks = np.empty_like(order)
    ranks[order] = np.arange(1, len(v) + 1)
    return ranks

def spearman_rank_corr(a: np.ndarray, b: np.ndarray) -> float:
    ra = rankdata_desc(a)
    rb = rankdata_desc(b)
    rho, _ = spearmanr(ra, rb)
    return float(rho)

def topk_indices(v: np.ndarray, k: int) -> np.ndarray:
    return np.argsort(-np.abs(v))[:k]

def ieee_axes(ax):
    # IEEE/TMLR-friendly: clean axes, readable ticks, no heavy styling
    ax.grid(True, which="both", alpha=0.25)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

def save_fig(fig, out_path_base: str, dpi: int = 220):
    fig.tight_layout()
    fig.savefig(out_path_base + ".png", dpi=dpi)
    fig.savefig(out_path_base + ".pdf")
    plt.close(fig)


# -----------------------------
# Dataset specs
# -----------------------------
@dataclass
class DatasetSpec:
    name: str
    target_col: str
    feature_cols: List[str]
    time_col: Optional[str] = None


# -----------------------------
# Fetch datasets
# -----------------------------
def fetch_uci_appliances(data_dir: str) -> Tuple[pd.DataFrame, DatasetSpec]:
    ensure_dir(data_dir)
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv"
    csv_path = os.path.join(data_dir, "energydata_complete.csv")
    if not os.path.exists(csv_path):
        print(f"[download] {url}")
        download_url(url, csv_path)

    df = pd.read_csv(csv_path)
    time_col = "date"
    target_col = "Appliances"
    df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
    df = df.dropna(subset=[time_col]).sort_values(time_col).reset_index(drop=True)
    feature_cols = [c for c in df.columns if c not in [time_col, target_col]]
    return df, DatasetSpec("uci_appliances", target_col, feature_cols, time_col)


def fetch_uci_household_power(data_dir: str) -> Tuple[pd.DataFrame, DatasetSpec]:
    ensure_dir(data_dir)
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip"
    zip_path = os.path.join(data_dir, "household_power_consumption.zip")
    if not os.path.exists(zip_path):
        print(f"[download] {url}")
        download_url(url, zip_path)

    txt_path = os.path.join(data_dir, "household_power_consumption.txt")
    if not os.path.exists(txt_path):
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(data_dir)

    df = pd.read_csv(txt_path, sep=";", na_values=["?"], low_memory=False)

    df["DateTime"] = pd.to_datetime(df["Date"] + " " + df["Time"], errors="coerce", dayfirst=True)
    df = df.dropna(subset=["DateTime"]).sort_values("DateTime").reset_index(drop=True)

    for c in df.columns:
        if c not in ["Date", "Time", "DateTime"]:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    df = df.dropna().reset_index(drop=True)

    target_col = "Global_active_power"
    time_col = "DateTime"
    feature_cols = [c for c in df.columns if c not in [target_col, time_col, "Date", "Time"]]
    return df, DatasetSpec("uci_household_power", target_col, feature_cols, time_col)


def fetch_uci_bikesharing_hourly(data_dir: str) -> Tuple[pd.DataFrame, DatasetSpec]:
    """
    Bike Sharing Dataset:
    https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
    We use hour.csv and predict cnt (total rentals).
    """
    ensure_dir(data_dir)
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip"
    zip_path = os.path.join(data_dir, "Bike-Sharing-Dataset.zip")
    if not os.path.exists(zip_path):
        print(f"[download] {url}")
        download_url(url, zip_path)

    hour_csv = os.path.join(data_dir, "hour.csv")
    if not os.path.exists(hour_csv):
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(data_dir)

    df = pd.read_csv(hour_csv)
    # 'dteday' is date (no hour). actual hour is 'hr'. Keep a constructed timestamp for ordering.
    df["dteday"] = pd.to_datetime(df["dteday"], errors="coerce")
    df = df.dropna(subset=["dteday"]).reset_index(drop=True)
    df["DateTime"] = df["dteday"] + pd.to_timedelta(df["hr"].astype(int), unit="h")
    df = df.sort_values("DateTime").reset_index(drop=True)

    target_col = "cnt"
    time_col = "DateTime"
    drop_cols = {"instant", "dteday", "DateTime", target_col, "casual", "registered"}  # avoid leakage cols
    feature_cols = [c for c in df.columns if c not in drop_cols]
    return df, DatasetSpec("uci_bikesharing_hourly", target_col, feature_cols, time_col)


def get_all_datasets(data_dir: str, which: str) -> List[Tuple[pd.DataFrame, DatasetSpec]]:
    """
    which: 'all' or comma-separated list among:
      uci_appliances, uci_household_power, uci_bikesharing_hourly
    """
    ds_map = {
        "uci_appliances": fetch_uci_appliances,
        "uci_household_power": fetch_uci_household_power,
        "uci_bikesharing_hourly": fetch_uci_bikesharing_hourly,
    }
    if which.strip().lower() == "all":
        keys = list(ds_map.keys())
    else:
        keys = [k.strip() for k in which.split(",") if k.strip()]
        for k in keys:
            if k not in ds_map:
                raise ValueError(f"Unknown dataset '{k}'. Allowed: {list(ds_map.keys())} or 'all'.")

    out = []
    for k in keys:
        df, spec = ds_map[k](data_dir)
        out.append((df, spec))
    return out


# -----------------------------
# Feature engineering (FAST, no fragmentation)
# -----------------------------
def make_supervised_lagged(df: pd.DataFrame, spec: DatasetSpec, lags: int, horizon: int) -> Tuple[pd.DataFrame, pd.Series, List[str]]:
    df = df.copy()
    if spec.time_col is not None:
        df = df.sort_values(spec.time_col).reset_index(drop=True)

    if spec.time_col is not None:
        t = pd.to_datetime(df[spec.time_col])
        base = df[spec.feature_cols].copy()
        base["hour"] = t.dt.hour
        base["dayofweek"] = t.dt.dayofweek
        base["month"] = t.dt.month
    else:
        base = df[spec.feature_cols].copy()

    lagged_blocks = []
    colnames = []
    for L in range(1, lags + 1):
        shifted = base.shift(L)
        shifted.columns = [f"{c}_lag{L}" for c in shifted.columns]
        lagged_blocks.append(shifted)
        colnames.extend(list(shifted.columns))

    X = pd.concat(lagged_blocks, axis=1)
    y = df[spec.target_col].shift(-horizon)

    valid = X.notnull().all(axis=1) & y.notnull()
    X = X.loc[valid].reset_index(drop=True)
    y = y.loc[valid].reset_index(drop=True)

    return X, y, colnames


# -----------------------------
# Models + SHAP
# -----------------------------
def train_tree_model(X_train: np.ndarray, y_train: np.ndarray, seed: int):
    model = HistGradientBoostingRegressor(
        max_depth=6,
        learning_rate=0.08,
        max_iter=300,
        random_state=seed
    )
    model.fit(X_train, y_train)
    return model

def shap_summary_vector(model, X_bg: np.ndarray, X_eval: np.ndarray) -> np.ndarray:
    """
    Robust SHAP summary for tree ensembles in Colab:
      - feature_perturbation='interventional' avoids leaf coverage error
      - disable additivity check at shap_values() call
    """
    explainer = shap.TreeExplainer(model, data=X_bg, feature_perturbation="interventional")
    shap_vals = explainer.shap_values(X_eval, check_additivity=False)
    shap_vals = np.asarray(shap_vals)
    return np.mean(np.abs(shap_vals), axis=0)


# -----------------------------
# HXAI mechanisms
# -----------------------------
def split_into_households_contiguous(X: np.ndarray, y: np.ndarray, k: int):
    n = len(X)
    block = n // k
    chunks = []
    for i in range(k):
        a = i * block
        b = (i + 1) * block if i < k - 1 else n
        chunks.append((X[a:b], y[a:b]))
    return chunks

def hxai_zonal_aggregate(local_summaries: List[np.ndarray], epsilons: List[float], sensitivity_l1: float) -> np.ndarray:
    d = local_summaries[0].shape[0]
    zonal = np.zeros(d, dtype=float)
    for Si, eps in zip(local_summaries, epsilons):
        scale = sensitivity_l1 / max(eps, 1e-12)
        zonal += Si + laplace_noise(scale=scale, size=d)
    return zonal

def negotiate_epsilons(local_summaries: List[np.ndarray], epsilon_total: float, eps_min: float, eps_max: float, temperature: float = 0.6):
    norms = np.array([np.linalg.norm(Si) for Si in local_summaries], dtype=float)
    norms = np.maximum(norms, 1e-12)
    w = (norms ** temperature)
    w = w / np.sum(w)

    eps = epsilon_total * w
    eps = np.clip(eps, eps_min, eps_max)

    # renormalize with clamps
    for _ in range(10):
        s = float(np.sum(eps))
        if s <= 1e-12:
            eps[:] = epsilon_total / len(eps)
            break
        eps *= (epsilon_total / s)
        eps = np.clip(eps, eps_min, eps_max)

    return eps.tolist()

def central_dp_baseline(X: np.ndarray, y: np.ndarray, sensitivity_l1: float, epsilon: float, seed: int):
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, shuffle=False)
    model = train_tree_model(Xtr, ytr, seed=seed)
    preds = model.predict(Xte)
    metrics = {"mse": float(mean_squared_error(yte, preds)), "r2": float(r2_score(yte, preds))}

    bg = Xtr[np.linspace(0, len(Xtr) - 1, num=min(512, len(Xtr))).astype(int)]
    ev = Xte[np.linspace(0, len(Xte) - 1, num=min(1024, len(Xte))).astype(int)]

    S = shap_summary_vector(model, bg, ev)
    scale = sensitivity_l1 / max(epsilon, 1e-12)
    return S + laplace_noise(scale=scale, size=S.shape[0]), metrics


# -----------------------------
# One run (one dataset, one seed)
# -----------------------------
def run_one_seed(
    df: pd.DataFrame,
    spec: DatasetSpec,
    lags: int,
    horizon: int,
    k_households: int,
    eps_grid: List[float],
    sensitivity_l1: float,
    seed: int,
) -> Tuple[pd.DataFrame, Dict]:
    set_seed(seed)

    Xdf, y, feat_cols = make_supervised_lagged(df, spec, lags=lags, horizon=horizon)
    X = Xdf.values.astype(float)
    yv = y.values.astype(float)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    chunks = split_into_households_contiguous(X, yv, k=k_households)

    local_summaries = []
    local_models = []
    local_bg_eval = []
    local_pred_metrics = []

    for i, (Xi, yi) in enumerate(chunks):
        Xtr, Xte, ytr, yte = train_test_split(Xi, yi, test_size=0.2, shuffle=False)
        model = train_tree_model(Xtr, ytr, seed=seed + i)

        preds = model.predict(Xte)
        local_pred_metrics.append((float(mean_squared_error(yte, preds)), float(r2_score(yte, preds))))

        bg = Xtr[np.linspace(0, len(Xtr) - 1, num=min(256, len(Xtr))).astype(int)]
        ev = Xte[np.linspace(0, len(Xte) - 1, num=min(512, len(Xte))).astype(int)]

        Si = shap_summary_vector(model, bg, ev)

        local_summaries.append(Si)
        local_models.append(model)
        local_bg_eval.append((bg, ev))

    # oracle zonal (no DP)
    S_true = np.sum(np.stack(local_summaries, axis=0), axis=0)

    rows = []
    d = X.shape[1]

    for eps in eps_grid:
        # HXAI uniform eps
        S_same = hxai_zonal_aggregate(local_summaries, [eps] * k_households, sensitivity_l1)

        # HXAI negotiated eps (same total budget)
        eps_total = k_households * eps
        eps_list = negotiate_epsilons(
            local_summaries=local_summaries,
            epsilon_total=eps_total,
            eps_min=max(0.05, 0.10 * eps),
            eps_max=10.0 * eps,
            temperature=0.6
        )
        S_neg = hxai_zonal_aggregate(local_summaries, eps_list, sensitivity_l1)

        # Central DP baseline
        S_central, central_metrics = central_dp_baseline(X, yv, sensitivity_l1, eps, seed=seed)

        for method, S_hat in [
            ("Central_DP_SHAP", S_central),
            ("HXAI_negotiated_eps", S_neg),
            ("HXAI_same_eps", S_same),
        ]:
            rows.append({
                "dataset": spec.name,
                "seed": seed,
                "method": method,
                "epsilon": eps,
                "utility_cosine": cosine_similarity(S_true, S_hat),
                "rank_spearman": spearman_rank_corr(S_true, S_hat),
                "mean_abs_error": float(np.mean(np.abs(S_true - S_hat))),
                "d_features": d,
                "k_households": k_households,
                "sensitivity_l1": sensitivity_l1,
                "local_avg_mse": float(np.mean([m for m, _ in local_pred_metrics])),
                "local_avg_r2": float(np.mean([r for _, r in local_pred_metrics])),
                "central_mse": central_metrics.get("mse", np.nan),
                "central_r2": central_metrics.get("r2", np.nan),
            })

    artifact = {
        "feat_cols": feat_cols,
        "S_true": S_true,
        "local_summaries": local_summaries,
        "k_households": k_households,
        "d": d
    }
    return pd.DataFrame(rows), artifact


# -----------------------------
# Plot curves with confidence bands (mean ± std across seeds)
# -----------------------------
def plot_curves_with_bands(res_agg: pd.DataFrame, out_dir: str, dataset_name: str):
    ensure_dir(out_dir)

    def _plot(metric: str, ylabel: str, fname: str):
        fig, ax = plt.subplots(figsize=(6.2, 4.2))
        for method in sorted(res_agg["method"].unique()):
            rr = res_agg[(res_agg["method"] == method) & (res_agg["dataset"] == dataset_name)].sort_values("epsilon")
            x = rr["epsilon"].values
            y = rr[f"{metric}_mean"].values
            s = rr[f"{metric}_std"].values
            ax.plot(x, y, marker="o", linewidth=2, label=method)
            ax.fill_between(x, y - s, y + s, alpha=0.18)

        ax.set_xscale("log")
        ax.set_xlabel(r"$\epsilon$ (log scale)")
        ax.set_ylabel(ylabel)
        ax.set_title(f"{dataset_name}: {ylabel} vs $\\epsilon$")
        ieee_axes(ax)
        ax.legend(frameon=True)
        save_fig(fig, os.path.join(out_dir, f"curves_{dataset_name}_{fname}"))

    _plot("utility_cosine", "Utility (cosine similarity)", "utility")
    _plot("rank_spearman", "Ranking stability (Spearman)", "rank")
    _plot("mean_abs_error", "Mean absolute error", "mae")


# -----------------------------
# Qualitative plots:
#   - local vs zonal SHAP bars (noiseless vs noisy)
#   - rank trajectories vs epsilon for top-k oracle features
# -----------------------------
def plot_local_zonal_bars(
    dataset_name: str,
    feat_cols: List[str],
    S_true: np.ndarray,
    local_summaries: List[np.ndarray],
    eps: float,
    sensitivity_l1: float,
    out_dir: str,
    top_k: int = 12,
    household_idx: int = 0,
):
    ensure_dir(out_dir)

    k_households = len(local_summaries)
    d = S_true.shape[0]
    idx = topk_indices(S_true, top_k)

    # oracle local (no noise)
    S_local = local_summaries[household_idx].copy()

    # local noisy
    scale_local = sensitivity_l1 / max(eps, 1e-12)
    S_local_noisy = S_local + laplace_noise(scale_local, d)

    # zonal noisy (uniform eps)
    S_zonal_noisy = hxai_zonal_aggregate(local_summaries, [eps] * k_households, sensitivity_l1)

    labels = [feat_cols[i] for i in idx]

    fig, axes = plt.subplots(1, 3, figsize=(12.6, 4.4), sharey=True)
    titles = [
        f"Local (oracle), h={household_idx}",
        f"Local (DP), h={household_idx}, eps={eps:g}",
        f"Zonal (DP), eps={eps:g}"
    ]
    vecs = [S_local, S_local_noisy, S_zonal_noisy]

    for ax, title, vec in zip(axes, titles, vecs):
        vals = vec[idx]
        ax.barh(np.arange(len(idx)), vals)
        ax.set_yticks(np.arange(len(idx)))
        ax.set_yticklabels(labels, fontsize=8)
        ax.invert_yaxis()
        ax.set_title(title)
        ieee_axes(ax)

    fig.suptitle(f"{dataset_name}: Local vs Zonal explanation (Top-{top_k})", y=1.02)
    save_fig(fig, os.path.join(out_dir, f"qual_{dataset_name}_bars_eps{eps:g}"))


def compute_rank_trajectory(S_true: np.ndarray, S_hat_list: List[np.ndarray]) -> np.ndarray:
    """
    Return trajectory ranks for each feature across eps:
      rank[t, j] = rank of feature j at eps[t] under S_hat[t], based on abs magnitude.
    """
    T = len(S_hat_list)
    d = S_true.shape[0]
    ranks = np.zeros((T, d), dtype=int)
    for t in range(T):
        ranks[t] = rankdata_desc(S_hat_list[t])
    return ranks  # ranks are 1..d


def plot_rank_trajectories(
    dataset_name: str,
    feat_cols: List[str],
    S_true: np.ndarray,
    eps_grid: List[float],
    S_hat_by_eps: Dict[str, List[np.ndarray]],
    out_dir: str,
    top_k: int = 10
):
    ensure_dir(out_dir)
    idx = topk_indices(S_true, top_k)
    top_labels = [feat_cols[i] for i in idx]

    # oracle ranks for reference
    oracle_ranks = rankdata_desc(S_true)
    oracle_top_ranks = oracle_ranks[idx]

    for method, S_list in S_hat_by_eps.items():
        ranks = compute_rank_trajectory(S_true, S_list)  # (T, d)
        fig, ax = plt.subplots(figsize=(6.6, 4.4))

        for f_i, feat_index in enumerate(idx):
            y = ranks[:, feat_index]
            ax.plot(eps_grid, y, marker="o", linewidth=1.8, label=f"{top_labels[f_i]} (oracle rank={oracle_top_ranks[f_i]})")

        ax.set_xscale("log")
        ax.set_xlabel(r"$\epsilon$ (log scale)")
        ax.set_ylabel("Rank (lower is better)")
        ax.set_title(f"{dataset_name}: Rank trajectories (Top-{top_k}) — {method}")
        ax.invert_yaxis()
        ieee_axes(ax)

        # Keep legend readable: place outside for many lines
        ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), fontsize=8, frameon=True)
        save_fig(fig, os.path.join(out_dir, f"qual_{dataset_name}_ranktraj_{method}"))


# -----------------------------
# Build noisy S_hat lists for qualitative rank plots (one seed, one dataset)
# -----------------------------
def build_noisy_explanations_over_eps(
    S_true: np.ndarray,
    local_summaries: List[np.ndarray],
    X_all: np.ndarray,
    y_all: np.ndarray,
    eps_grid: List[float],
    sensitivity_l1: float,
    seed: int
) -> Dict[str, List[np.ndarray]]:
    set_seed(seed)

    k_households = len(local_summaries)

    S_same_list = []
    S_neg_list = []
    S_cent_list = []

    for eps in eps_grid:
        S_same = hxai_zonal_aggregate(local_summaries, [eps] * k_households, sensitivity_l1)

        eps_total = k_households * eps
        eps_list = negotiate_epsilons(
            local_summaries=local_summaries,
            epsilon_total=eps_total,
            eps_min=max(0.05, 0.10 * eps),
            eps_max=10.0 * eps,
            temperature=0.6
        )
        S_neg = hxai_zonal_aggregate(local_summaries, eps_list, sensitivity_l1)

        S_cent, _ = central_dp_baseline(X_all, y_all, sensitivity_l1, eps, seed=seed)

        S_same_list.append(S_same)
        S_neg_list.append(S_neg)
        S_cent_list.append(S_cent)

    return {
        "HXAI_same_eps": S_same_list,
        "HXAI_negotiated_eps": S_neg_list,
        "Central_DP_SHAP": S_cent_list
    }


# -----------------------------
# Main
# -----------------------------
def main(args):
    ensure_dir(args.out_dir)
    ensure_dir(args.data_dir)

    # eps grid (log-spaced)
    eps_grid = np.logspace(math.log10(args.eps_min), math.log10(args.eps_max), args.eps_points).tolist()

    # seeds list
    seeds = [int(s) for s in args.seeds.split(",") if s.strip()]

    # datasets
    datasets = get_all_datasets(args.data_dir, args.datasets)

    all_rows = []

    # For qualitative plots, store one "representative seed artifact" per dataset (first seed)
    qual_artifacts = {}

    for (df, spec) in datasets:
        print(f"\n[dataset] {spec.name}  n={len(df)}  d_raw={len(spec.feature_cols)}  time={spec.time_col}")
        for si, seed in enumerate(seeds):
            res_seed, artifact = run_one_seed(
                df=df, spec=spec,
                lags=args.lags, horizon=args.horizon,
                k_households=args.k_households,
                eps_grid=eps_grid,
                sensitivity_l1=args.sensitivity_l1,
                seed=seed
            )
            all_rows.append(res_seed)

            if si == 0:
                # For qual plots we also need X_all / y_all aligned with lags
                Xdf, y, feat_cols = make_supervised_lagged(df, spec, args.lags, args.horizon)
                X_all = StandardScaler().fit_transform(Xdf.values.astype(float))
                y_all = y.values.astype(float)

                qual_artifacts[spec.name] = {
                    "seed": seed,
                    "feat_cols": artifact["feat_cols"],
                    "S_true": artifact["S_true"],
                    "local_summaries": artifact["local_summaries"],
                    "eps_grid": eps_grid,
                    "X_all": X_all,
                    "y_all": y_all
                }

    res_all = pd.concat(all_rows, axis=0).reset_index(drop=True)
    per_seed_csv = os.path.join(args.out_dir, "results_per_seed.csv")
    res_all.to_csv(per_seed_csv, index=False)
    print(f"\n[saved] {per_seed_csv}")

    # Aggregate mean/std across seeds
    agg = (
        res_all
        .groupby(["dataset", "method", "epsilon"], as_index=False)
        .agg(
            utility_cosine_mean=("utility_cosine", "mean"),
            utility_cosine_std=("utility_cosine", "std"),
            rank_spearman_mean=("rank_spearman", "mean"),
            rank_spearman_std=("rank_spearman", "std"),
            mean_abs_error_mean=("mean_abs_error", "mean"),
            mean_abs_error_std=("mean_abs_error", "std"),
            d_features=("d_features", "first"),
            k_households=("k_households", "first"),
            sensitivity_l1=("sensitivity_l1", "first")
        )
    )
    agg_csv = os.path.join(args.out_dir, "results_agg_meanstd.csv")
    agg.to_csv(agg_csv, index=False)
    print(f"[saved] {agg_csv}")

    # Curves with bands
    for (df, spec) in datasets:
        plot_curves_with_bands(agg, args.out_dir, spec.name)

    # Qualitative plots (bars + rank trajectories) for each dataset using representative seed
    for ds_name, art in qual_artifacts.items():
        feat_cols = art["feat_cols"]
        S_true = art["S_true"]
        local_summaries = art["local_summaries"]
        eps_grid = art["eps_grid"]
        X_all = art["X_all"]
        y_all = art["y_all"]
        seed = art["seed"]

        # choose eps values to visualize (closest in grid)
        def closest_eps(target: float):
            arr = np.array(eps_grid)
            return float(arr[np.argmin(np.abs(arr - target))])

        eps_low = closest_eps(args.qual_eps_low)
        eps_mid = closest_eps(args.qual_eps_mid)

        plot_local_zonal_bars(
            dataset_name=ds_name,
            feat_cols=feat_cols,
            S_true=S_true,
            local_summaries=local_summaries,
            eps=eps_low,
            sensitivity_l1=args.sensitivity_l1,
            out_dir=args.out_dir,
            top_k=args.qual_top_k,
            household_idx=min(args.qual_household_idx, len(local_summaries) - 1)
        )
        plot_local_zonal_bars(
            dataset_name=ds_name,
            feat_cols=feat_cols,
            S_true=S_true,
            local_summaries=local_summaries,
            eps=eps_mid,
            sensitivity_l1=args.sensitivity_l1,
            out_dir=args.out_dir,
            top_k=args.qual_top_k,
            household_idx=min(args.qual_household_idx, len(local_summaries) - 1)
        )

        # Rank trajectories across eps for each method
        S_hat_by_eps = build_noisy_explanations_over_eps(
            S_true=S_true,
            local_summaries=local_summaries,
            X_all=X_all,
            y_all=y_all,
            eps_grid=eps_grid,
            sensitivity_l1=args.sensitivity_l1,
            seed=seed
        )
        plot_rank_trajectories(
            dataset_name=ds_name,
            feat_cols=feat_cols,
            S_true=S_true,
            eps_grid=eps_grid,
            S_hat_by_eps=S_hat_by_eps,
            out_dir=args.out_dir,
            top_k=args.rank_top_k
        )

    # Quick textual summary
    print("\nQuick check (best utility per dataset/method):")
    best = (
        agg.sort_values("utility_cosine_mean", ascending=False)
           .groupby(["dataset", "method"], as_index=False)
           .head(1)[["dataset", "method", "epsilon", "utility_cosine_mean", "rank_spearman_mean", "mean_abs_error_mean"]]
    )
    print(best.to_string(index=False))

    print("\nDone. Check outputs in:", args.out_dir)


def build_argparser():
    ap = argparse.ArgumentParser()
    ap.add_argument("--out_dir", type=str, default="hxai_realdata_out")
    ap.add_argument("--data_dir", type=str, default="hxai_realdata_data")

    ap.add_argument("--datasets", type=str, default="all",
                    help="all OR comma-separated: uci_appliances,uci_household_power,uci_bikesharing_hourly")

    ap.add_argument("--k_households", type=int, default=20)
    ap.add_argument("--lags", type=int, default=6)
    ap.add_argument("--horizon", type=int, default=1)

    ap.add_argument("--eps_min", type=float, default=0.1)
    ap.add_argument("--eps_max", type=float, default=20.0)
    ap.add_argument("--eps_points", type=int, default=10)

    ap.add_argument("--sensitivity_l1", type=float, default=1.0)

    ap.add_argument("--seeds", type=str, default="42,43,44",
                    help="comma-separated seeds for mean±std bands")

    # qualitative plots
    ap.add_argument("--qual_eps_low", type=float, default=0.2)
    ap.add_argument("--qual_eps_mid", type=float, default=1.0)
    ap.add_argument("--qual_top_k", type=int, default=12)
    ap.add_argument("--qual_household_idx", type=int, default=0)

    ap.add_argument("--rank_top_k", type=int, default=10)

    return ap


if __name__ == "__main__":
    parser = build_argparser()
    # Jupyter/Colab safe
    args, _unknown = parser.parse_known_args()
    main(args)


[download] https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv
[download] https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip
[download] https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip

[dataset] uci_appliances  n=19735  d_raw=27  time=date


 96%|=================== | 984/1024 [00:17<00:00]       


[dataset] uci_household_power  n=2049280  d_raw=6  time=DateTime


 96%|=================== | 980/1024 [00:11<00:00]       


[dataset] uci_bikesharing_hourly  n=17379  d_raw=12  time=DateTime


 96%|=================== | 985/1024 [00:16<00:00]       


[saved] hxai_realdata_out/results_per_seed.csv
[saved] hxai_realdata_out/results_agg_meanstd.csv


 94%|=================== | 965/1024 [00:16<00:00]       